In [1]:
#关于文档的一个数据分类
import pandas as pd
import seaborn as sns  #用于绘制热图的工具包
from scipy.cluster import hierarchy  #用于进行层次聚类，话层次聚类图的工具包
from scipy import cluster   
import matplotlib.pyplot as plt
from sklearn import decomposition as skldec #用于主成分分析降维的包
import numpy as np
#import csv
import math
from pandas import Series,DataFrame
from matplotlib.ticker import  MultipleLocator
import pylab as pl   #画图用
from sklearn import svm
from sklearn import linear_model

In [2]:
data = pd.read_excel('E:/vcdata/data/ALL_Intersection.xlsx')

In [3]:
### 对城市进行分类，大城市记为1，非大城市记为0.
pop17 = data['2017Pop'].values
data['popType']=pop17
for i in range(len(pop17)):
        if pop17[i] < 550:
            data['popType'][i]=1###小城市
        else:
            data['popType'][i]=0###特大城市

D:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
### 对城市进行分类，大城市记为1，非大城市记为0.
pop17 = data['2017Pop'].values
data['popType_0']=pop17
data['popType_1']=pop17
for i in range(len(pop17)):
        if pop17[i] < 550:
            data['popType_0'][i]=1###小城市
            data['popType_1'][i]=0###小城市
        else:
            data['popType_0'][i]=0###特大城市
            data['popType_1'][i]=1###特大城市

D:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
D:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
D:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A

In [35]:
a = np.arange(2000,2018)
pop = [str(i)+'Pop' for i in a]
investment = [str(i)+'总投资金额(M)' for i in a]
inv_times = [str(i)+'总投资次数' for i in a]
gdp = [str(i)+'GDP' for i in a]

In [36]:
### 获得双对数的值 ，舍掉双对数中的0值###
def get_log3(inv,g,DATA,name):
    ### + 1e-5解决RuntimeWarning: divide by zero encountered in log10 This is separate from the ipykernel package so we can avoid doing imports until
#     y_investment=np.log10(DATA[inv]+ 1e-5)
#     y_gdp=np.log10(DATA[g]+ 1e-5)
    y_investment=np.log10(DATA[inv])
    y_gdp=np.log10(DATA[g])
    y_investment2=y_investment.fillna(np.infty)
    y_gdp2=y_gdp.fillna(np.infty)
    types = DATA[name]
    x1=[]
    y1=[]
    Types = []
    for xi in range(len(y_investment2)):
        if y_investment2[xi]==-float('Inf') or y_investment2[xi]==float('Inf') or y_gdp2[xi]==-float('Inf') or y_gdp2[xi]==float('Inf'):
#             print('inf...')
            continue
#         elif xi>-4:
        else:
            x1.append(y_investment2[xi])
            y1.append(y_gdp2[xi])
            Types.append(types[xi]*y_investment2[xi])
    return(x1,y1,Types)

In [37]:
### 6.最小二乘拟合
from scipy.optimize import leastsq  ##引入最小二乘法算法
from sklearn.metrics import r2_score
#需要拟合的函数func :指定函数的形状
def func1(p,x,x2):
    k,b,c=p
    return k*x+b+c*x2
def error1(p,x,y,x2,s):
    return func1(p,x,x2)-y
def pic6(yt):
    yt1=yt[0]
    yt2=yt[1]
    yt3=yt[2]
    #k,b的初始值，可以任意设定,经过几次试验，发现p0的值会影响cost的值：Para[1]
    p0=[1,0,10]
    Xi=np.array(yt1)
    Yi=np.array(yt2)
    I=np.array(yt3)
    S='THE'
    #把error函数中除了p0以外的参数打包到args中(使用要求)
    Para=leastsq(error1,p0,args=(Xi,Yi,I,S))## 三个参数：误差函数、函数参数列表、数据点

    #读取结果
    k,b,c=Para[0]
    y = []
    for mm in range(len(yt1)):
        y.append(k*yt1[mm]+b+c*yt3[mm])
    y = [round(k*x+b+c*x,2) for x in yt1]
    k1=round(k, 2)
    r=r2_score(yt2,y)
    r2=round(r, 2)
    c=round(c, 2)
    ### 检测显著性
    #方差
    sigma2=sum((y-beta0-beta1*x)**2)/(18)
    #标准差
    sigma=np.sqrt(sigma2)
    #求t值
    t=beta1*np.sqrt(x2)/sigma
    print('t=',t)
    #已知临界值求p值
    p=stats.t.sf(t,18)
    print('p=',p)

    #输出检验结果
    if p<0.05:
        print ('the linear regression between x and y is significant')
    else:
        print('the linear regression between x and y is not significant')
    return(k1,b,c,y)

In [38]:
### 6.最小二乘拟合
from scipy.optimize import leastsq  ##引入最小二乘法算法
from sklearn.metrics import r2_score
#需要拟合的函数func :指定函数的形状
def func(p,x):
    k,b=p
    return k*x+b

##偏差函数：x,y都是列表:这里的x,y更上面的Xi,Yi中是一一对应的
def error(p,x,y):
    return func(p,x)-y
def pic(yt):
    yt1=yt[0]
    yt2=yt[1]
    #k,b的初始值，可以任意设定,经过几次试验，发现p0的值会影响cost的值：Para[1]
    p0=[1,10]
    Xi=np.array(yt1)
    Yi=np.array(yt2)
    #把error函数中除了p0以外的参数打包到args中(使用要求)
    Para=leastsq(error,p0,args=(Xi,Yi))## 三个参数：误差函数、函数参数列表、数据点

    #读取结果
    k,b=Para[0]
    y = [round(k*x+b,2) for x in yt1]
    k1=round(k, 2)
    r=r2_score(yt2,y)
    r2=round(r, 2)
    .pvalues
    return(k1)


In [68]:
######## 所有年份所有数据分类，大城市为1，其他城市为0
POP = []### 所有城市所有年份的人口
AMount = []###所有城市所有年份的投资金额
TYPE0 = []
TYPE1=[]
TimesB = []
GDPB = []
for i in range(18):
    X11=pop[i]
    X21=investment[i]
    X22=inv_times[i]
    X23=gdp[i]
    Y1 = get_log3(X11,X21,data,'popType_0')
    Y2 = get_log3(X11,X21,data,'popType_1')
    Y22 = get_log3(X11,X21,data,'popType_1')
    Y23 = get_log3(X11,X21,data,'popType_1')
    POP += Y1[0]
    AMount += Y1[1]
    TYPE0 += Y1[2]
    TYPE1 += Y2[2]
    TimesB += Y22[1]
    GDPB += Y23[1]
#     Y_total = get_log(X11,X21,data)

D:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log10
  import sys


In [45]:
alpha_big0 = pic6([POP,AMount,TYPE0])[0]
gama_big0 = pic6([POP,AMount,TYPE0])[2]
alpha_total = pic([POP,AMount])

alpha_big1 = pic6([POP,AMount,TYPE1])[0]
gama_big1 = pic6([POP,AMount,TYPE1])[2]
alpha_total2 = pic([POP,AMount,TYPE1][0:2])

In [46]:
alpha_big1,gama_big1,alpha_total,alpha_big0,gama_big0,alpha_total

(0.93, 0.14, 1.14, 1.07, -0.14, 1.14)

In [43]:
[0]*2

[0, 0]

In [69]:
######## 所有年份所有数据分类，大城市为1，其他城市为0
POP = []### 所有城市所有年份的人口
AMount = []###所有城市所有年份的投资金额
TYPEyear0 = []
TYPEyear1=[]
Times = []
GDP = []
a = np.arange(2000,2018)
yearType = [str(i)+'yearType0' for i in a]### 2013年之前的为0
yearType2 = [str(i)+'yearType1' for i in a]###2013年之前的为1
for i in range(18):
    if i<13:
        data[yearType[i]] = [0]*data.shape[0]
        data[yearType2[i]] = [1]*data.shape[0]
    else:
        data[yearType[i]] = [1]*data.shape[0]
        data[yearType2[i]] = [0]*data.shape[0]
    X11=pop[i]
    X21=investment[i]
    X22=inv_times[i]
    X23=gdp[i]
    Y1 = get_log3(X11,X21,data,yearType[i])
    Y2 = get_log3(X11,X21,data,yearType2[i])
    Y12 = get_log3(X11,X22,data,yearType[i])
    Y13 = get_log3(X11,X23,data,yearType2[i])
    POP += Y1[0]
    AMount += Y1[1]
    TYPEyear0 += Y1[2]
    TYPEyear1 += Y2[2]
    Times += Y12[1]
    GDP += Y13[1]
#     Y_total = get_log(X11,X21,data)

D:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log10
  import sys


In [51]:
alpha_year0 = pic6([POP,AMount,TYPEyear0])[0]
gama_year0 = pic6([POP,AMount,TYPEyear0])[2]
alpha_total = pic([POP,AMount])

alpha_year1 = pic6([POP,AMount,TYPEyear1])[0]
gama_year1 = pic6([POP,AMount,TYPEyear1])[2]
alpha_total2 = pic([POP,AMount,TYPEyear1][0:2])

In [52]:
alpha_total2

1.14

In [58]:
df = pd.DataFrame()
df['alpha_total'] = [alpha_total]
df['alpha_big0'] =alpha_big0
df['gama_big0'] = gama_big0
df['alpha_big1'] = alpha_big1
df['gama_big1'] = gama_big1
df['alpha_year0'] =alpha_year0
df['gama_year0'] = gama_year0
df['alpha_year1'] = alpha_year1
df['gama_year1'] = gama_year1

In [59]:
df

,alpha_total,alpha_big0,gama_big0,alpha_big1,gama_big1,alpha_year0,gama_year0,alpha_year1,gama_year1
0,1.14,1.07,-0.14,0.93,0.14,1.17,-0.07,1.11,0.07


In [55]:
alpha_total

1.14

In [60]:
pop[12]

'2012Pop'

In [61]:
###############

In [62]:
d = []
d2 = []
for i in range(len(POP)):
    c = []
    c2 = []
    c.append(POP[i])
    c.append(TYPEyear1[i])
    d.append(c)
    c2.append(POP[i])
    c2.append(TYPE1[i])
    d2.append(c2)


In [63]:
regr = linear_model.LinearRegression()
regr.fit(d,AMount)
k = regr.coef_
b = regr.intercept_
pred = regr.predict(d)

In [64]:
pred

array([3.20294569, 3.16527814, 2.54041273, ..., 1.38525495, 1.49331599,
       1.53693479])

In [65]:
import statsmodels.api as sm
mod = sm.OLS(Y,X)
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']

(array([1.1076989 , 0.06638529]), -0.2867848132231492)

In [79]:
import sklearn
from sklearn import linear_model
lm = sklearn.linear_model.LinearRegression()
lm.fit(d,AMount)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
##########################################################

In [66]:
######### 得到均值这些数据 #########

In [70]:
#### 计算所有年amount、times、GDP的均值之类的指标
def getmean(amount):
    df = pd.DataFrame()
    df['amount'] = amount
    size = (df.shape[0] - (df == 0).astype(int).sum(axis=0)).values
    mean = df.mean(axis=0).values
    SD = df.std(axis=0).values
    maxvalues = df.max(axis=0).values
    minvalues = df.min(axis=0).values
    return(size,mean,SD,maxvalues,minvalues)

In [74]:
dataAmount = getmean(AMount)
dataTimes = getmean(Times)
dataGDP = getmean(GDP)

In [73]:
dataAmount = getmean(AMount)
dataTimesB = getmean(TimesB)
dataGDPB = getmean(GDPB)

In [75]:
dataTimes,dataGDP,

((array([1357], dtype=int64),
  array([0.56914928]),
  array([0.58867056]),
  array([3.25767857]),
  array([0.])),
 (array([4790], dtype=int64),
  array([6.47905135]),
  array([0.58227495]),
  array([8.48618939]),
  array([5.01639395])))

In [76]:
dataTimesB,dataGDPB

((array([1772], dtype=int64),
  array([2.07630025]),
  array([0.87196514]),
  array([5.03333742]),
  array([-0.76654698])),
 (array([1772], dtype=int64),
  array([2.07630025]),
  array([0.87196514]),
  array([5.03333742]),
  array([-0.76654698])))